## Homework 10
### Question Answering with Information Retrieval

Welcome to Homework 10! 

The homework contains several tasks. You can find the amount of points that you get for the correct solution in the task header. Maximum amount of points for each homework is _four_.

The **grading** for each task is the following:
- correct answer - **full points**
- insufficient solution or solution resulting in the incorrect output - **half points**
- no answer or completely wrong solution - **no points**

Even if you don't know how to solve the task, we encourage you to write down your thoughts and progress and try to address the issues that stop you from completing the task.

When working on the written tasks, try to make your answers short and accurate. Most of the times, it is possible to answer the question in 1-3 sentences.

When writing code, make it readable. Choose appropriate names for your variables (`a = 'cat'` - not good, `word = 'cat'` - good). Avoid constructing lines of code longer than 100 characters (79 characters is ideal). If needed, provide the commentaries for your code, however, a good code should be easily readable without them :)

Finally, all your answers should be written only by yourself. If you copy them from other sources it will be considered as an academic fraud. You can discuss the tasks with your classmates but each solution must be individual.

<font color='red'>**Important!:**</font> **before sending your solution, do the `Kernel -> Restart & Run All` to ensure that all your code works.**

In [ ]:
!pip install -U sentence-transformers rank_bm25 transformers

Download the Simple English Wikipedia. It has about 170,000 articles, each split into paragraphs.

In [1]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

Unpack the data and read all the paragraphs.

In [2]:
passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        passages.extend(data['paragraphs'])

print("Passages:", len(passages))

Passages: 509663


passages## Task 1. Information Retrieval with BM25 (1 point)

To build an open-domain QA system, we usually require two components: retriever and reader. Retriever, finds the passages that correspond best to the question, and reader extracts the question from them. 

In this task, you are going to build a retriever based on [BM25 algorithm](https://en.wikipedia.org/wiki/Okapi_BM25). It is a bag-of-words algorithm that is similar to TfIdf but usually more robust and accurate.

Use [`rank_bm25`](https://github.com/dorianbrown/rank_bm25) package to build a BM25 index on the extracted packages.

First, you will need to build a tokenizer that will lowercase the text, split into tokens and remove stopwords. Then, you tokenized the passages and build a BM25 index (for example `BM25Okapi`).

In [3]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    ### YOUR CODE HERE
    tokenized_doc=text.split( )
    tokenized_doc=[word.lower() for word in tokenized_doc]
    tokenized_doc=[word for word in tokenized_doc if not word in _stop_words.ENGLISH_STOP_WORDS]
    ### YOUR CODE HERE
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/509663 [00:00<?, ?it/s]

Next, make a function that will take a query (i.e. a question) and return top-k most similar passages along with their similarity scores. Refer to the [`rank_bm25` page](https://github.com/dorianbrown/rank_bm25) for more information. 

In [4]:
def bm25_retriever(bm25_model, query, top_k=3):
    ### YOUR CODE HERE
    tokenized_query = query.split( )
    return bm25_model.get_top_n(tokenized_query,passages,n=top_k)
    ### YOUR CODE HERE

Try to test it with different queries.

In [5]:
queries=["who smacks Chris Rock?","Which is your axe?","Who won the election of French president?"]

for query in queries:
    print("\n\n======================\n\n")
    print("Query:", query)
    print(bm25_retriever(bm25, query))





Query: who smacks Chris Rock?
['Seminole bats are insectivores. They eat large amounts of Hymenoptera (ants, bees and wasps), Coleoptera (beetles), Lepidoptera (moths). They have also been shown to eat smaller amounts of Homoptera (cicadas) and Diptera (flies).', 'After he was set free as prisoner of war, his hometown was made Polish. He found a new home in the Ruhr area. Then he studied laws in Cologne and Bonn.', 'Since they are an animated group, they cannot perform live, like other bands can. At Gorillaz concerts, the real members of the band stand behind screens which show computer-generated images of the animated group performing.']




Query: Which is your axe?
['Seminole bats are insectivores. They eat large amounts of Hymenoptera (ants, bees and wasps), Coleoptera (beetles), Lepidoptera (moths). They have also been shown to eat smaller amounts of Homoptera (cicadas) and Diptera (flies).', 'After he was set free as prisoner of war, his hometown was made Polish. He found a n

## Task 2. Information Retrieval with Bi-Encoder and CrossEncoder (2.5 points)

BM25 gives plausible results but it still underperforms compared to the state of the art Transformer-based models.

For the information retrieving, Bi-Encoder and CrossEncoder are usually used. You can read more about them here: https://www.sbert.net/examples/applications/cross-encoder/README.html.

In short, to compare two texts, Bi-Encoder encodes both of them with the same encoder separately before computing the cosine similarity. This allows to pre-compute the embeddings for the whole corpus and find similar texts very quickly. However, these models are usually less accurate then CrossEncoders.

Cross Encoder takes both sentences at once and then computes the similarity score based on the resulting representation. This leads to more accurate results but it reduces the inference computation speed since we have to encode each pair of texts separately.

Usually, these two encoders are used in tandem. First, you extract top-k similar texts with Bi-Encoder, and then rerank them with Cross Encoder to get more accurate results.

First, let's initiallize both of them using the [sentence-transformers](https://www.sbert.net/index.html) library.

In [6]:
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 32 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Next, precompute the embeddings for each paragraph in our corpus with Bi-Encoder. 

Note: This might take up to 40-50 minutes!

In [7]:
corpus_embeddings = bi_encoder.encode(passages, batch_size=128, convert_to_tensor=True, show_progress_bar=True)

# Optionally save the embeddings to load them faster next time
# torch.save(corpus_embeddings, 'corpus_embeddings.pt')

Batches:   0%|          | 0/3982 [00:00<?, ?it/s]

Next, create a function that will use a Bi-Encoder to find top-k similar texts in the corpus and return them with their scores.

For this, first [encode the query with the Bi-Encoder](https://www.sbert.net/docs/package_reference/SentenceTransformer.html#sentence_transformers.SentenceTransformer.encode). Then, transfer the resulting embedding to the GPU. Finally, use the [`util.semantic_search`](https://www.sbert.net/docs/package_reference/util.html#sentence_transformers.util.semantic_search) to find the most similar texts.

In [8]:
def bi_encoder_retriever(bi_encoder, query, corpus_embeddings, top_k=32):
    ### YOUR CODE HERE
    bi_encoder_output=[]
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=top_k)
    for hit in hits[0]:
        bi_encoder_output.append(passages[hit['corpus_id']])
    
    return bi_encoder_output
    ### YOUR CODE HERE

Try to test it with different queries.

In [9]:
queries=["who smacks Chris Rock?","Which is your axe?","Who won the election of French president?"]

for query in queries:
    bi_encoder_output = bi_encoder_retriever(bi_encoder, query, corpus_embeddings)
    print("\n\n======================\n\n")
    print("Query:", query)
    print(bi_encoder_output)





Query: who smacks Chris Rock?
['Chris Rock (born February 7, 1965) is an American comedian, actor, producer, screenwriter and director. He was born in Andrews, South Carolina and raised in Brooklyn, New York City. He is known for his jokes about racial stereotypes; he talks a lot about differences that are common between black people and white people in the United States. Rock has been in many television shows, such as "Everybody Hates Chris" and movies, such as "Dogma".', 'Chris Thile () (born February 20, 1981) is an American musician, best known as a member of acoustic band Nickel Creek. He has made six albums as a solo artist and with his band, Punch Brothers. His first, "Leading Off", was released in 1994 when Thile was 13. Thile has also played and recorded with artists like Mike Marshall, Béla Fleck, Glen Phillips, and Edgar Meyer.', 'Christopher Adam "Chris" Daughtry (born December 26, 1979) is an American singer. He is best known for being the fourth-place finisher on "Ame

Now, make a function that would take the retrieved texts by the Bi-Encoder and [re-rank them with the CrossEncoder](https://www.sbert.net/examples/applications/cross-encoder/README.html#cross-encoders-usage).

In [10]:
def rerank(cross_encoder, query, bi_encoder_output):
    ### YOUR CODE HERE
    rerank=[]
    sentence_combinations = [[query, corpus_sentence] for corpus_sentence in bi_encoder_output]
    similarity_scores = cross_encoder.predict(sentence_combinations)
    sim_scores_argsort = reversed(np.argsort(similarity_scores))
    
    for idx in sim_scores_argsort:
        rerank.append(bi_encoder_output[idx])
    return rerank
    ### YOUR CODE HERE

Test the rerank function.

In [11]:
queries=["who smacks Chris Rock?","Which is your axe?","Who won the election of French president?"]

for query in queries:
    cross_encoder_output = rerank(cross_encoder, query, bi_encoder_output)
    print("\n\n======================\n\n")
    print("Query:", query)
    print(cross_encoder_output)





Query: who smacks Chris Rock?
["He was the lead candidate for the Workers' Party in the 1994 European Parliament election; he later received 0.43% of the vote. He ran for president in the 2002 French presidential election. He received 0.47% of the vote, with 132,686 votes.", 'In the 2004 and 2008 elections, he won in Montmorency—Charlevoix—Haute-Côte-Nord before being defeated in the 2011 federal election. A lawyer, he has served as the Bloc critic of Parliamentary Affairs, Transport and to the Auditor General.', 'He was elected to the French Academy of Sciences the 18th of November 2003.', 'Alain Marie Juppé (; born 15 August 1945) is a French politician. He was a member of The Republicans. He was Prime Minister of France from 1995 to 1997 under President Jacques Chirac. He was President of the political party Union for a Popular Movement (UMP) from 2002 to 2004 and mayor of Bordeaux from 1995 to 2004.', 'He served in the French National Assembly from 1956-1959.', 'He was elected 

## Task 3. Build a QA pipeline (0.5 points)

Now, we are ready to build the final QA system.

First, let's load the QA pipeline from `transformers`. It takes the question and the context as inputs and produces the answer and its score. More info here: https://huggingface.co/docs/transformers/v4.18.0/en/task_summary#extractive-question-answering.

In [12]:
from transformers import pipeline

qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


First, build a function that will first retrieve top-10 similar passages with BM25 algorithm and then predicts the answer and the score for each of them using the `qa_model`. The score of the final answer is the average score produced by the retriever and reader.

In [25]:
### YOUR CODE HERE
question="who smacks Chris Rock?"
def mb25_qa_pipeline(question,top_k=10):
    bm25_context=bm25_retriever(bm25, query,top_k=10)
    result = qa_model(question=question, context=bm25_context[0])
    return f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"

mb25_qa_pipeline(question)

"Answer: 'Truman', score: 0.9813, start: 0, end: 6"

Then, do the same but this time use the Bi-Encoder as the retriever.

In [26]:
### YOUR CODE HERE
question="who smacks Chris Rock?"
def bi_encoder_qa_pipeline(question,top_k=10):
    bi_encoder_output=bi_encoder_retriever(bi_encoder, question, corpus_embeddings,top_k=10)
    bi_encoder_context=' '.join(context for context in bi_encoder_output)
    result = qa_model(question=question, context=bi_encoder_context)
    return f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"

bi_encoder_qa_pipeline(question)

"Answer: 'Robert Jens Rock', score: 0.5509, start: 1056, end: 1072"

Finally, do the same as above but this time use the re-ranked passages from the Cross Encoder as the retriever.

In [27]:
### YOUR CODE HERE
question="who smacks Chris Rock?"

def cross_encoder_qa_pipeline(question,top_k=10):
    bi_encoder_output=bi_encoder_retriever(bi_encoder, question, corpus_embeddings,top_k=10)
    cross_encoder_context=rerank(cross_encoder, question, bi_encoder_output)
    cross_encoder_context=' '.join(context for context in cross_encoder_context)
    result = qa_model(question=question, context=cross_encoder_context)
    return f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"

cross_encoder_qa_pipeline(question)

"Answer: 'Christopher James Hardman', score: 0.7294, start: 444, end: 469"

Try to answer different questions with all three systems and compare the answers. Which system is the most accurate?

In [31]:
### YOUR CODE HERE
queries=["who smacks Chris Rock?","Which is your axe?","Who won the election of French president?"]

for query in queries:
    print("\n\n======================\n\n")
    print("Query:", query)
    print("BM25 algorithm:", mb25_qa_pipeline(query))
    print("Bi-Encoder    :", bi_encoder_qa_pipeline(query))
    print("Cross Encoder :", cross_encoder_qa_pipeline(query))






Query: who smacks Chris Rock?
BM25 algorithm: Answer: 'Seminole bats', score: 0.3605, start: 0, end: 13
Bi-Encoder    : Answer: 'Robert Jens Rock', score: 0.5509, start: 1056, end: 1072
Cross Encoder : Answer: 'Christopher James Hardman', score: 0.7294, start: 444, end: 469




Query: Which is your axe?
BM25 algorithm: Answer: 'Seminole bats are insectivores', score: 0.0217, start: 0, end: 30
Bi-Encoder    : Answer: 'battle-axe', score: 0.4455, start: 78, end: 88
Cross Encoder : Answer: 'Arkalochori Axe', score: 0.7956, start: 979, end: 994




Query: Who won the election of French president?
BM25 algorithm: Answer: 'Truman', score: 0.995, start: 0, end: 6
Bi-Encoder    : Answer: 'Nicolas Sarkozy', score: 0.8434, start: 1546, end: 1561
Cross Encoder : Answer: 'François Hollande', score: 0.3893, start: 134, end: 151


**(A) :** According to result above, Cross Encoder has a good score, but in the last question has worst result.